# Build Chatbot
`11_chatbot.ipynb`
- https://python.langchain.com/docs/tutorials/chatbot/

In [1]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()

True

In [2]:
from langchain_core.messages import HumanMessage, AIMessage

messages = [
    HumanMessage(content='Hi!, I am bob.'),
    AIMessage(content='Hello bob. how can I help you.'),
    HumanMessage(content='Say my name.')
]

llm = ChatOpenAI(model='gpt-4.1-nano', temperature=0)

res = llm.invoke(messages)

res.pretty_print()

================================== Ai Message ==================================

Hi, Bob! Nice to meet you.


In [3]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, END, MessagesState, StateGraph

# Graph Builider
builder = StateGraph(state_schema=MessagesState)

# Node
def simple_node(state: MessagesState):
    res = llm.invoke(state['messages'])
    return {'messages': res}

builder.add_node('simple_node', simple_node)

# Edge (Node 끼리 연결)
builder.add_edge(START, 'simple_node')
builder.add_edge('simple_node', END)

# Memory (대화내역 기록)
memory = MemorySaver()

# Graph (그래프 생성)
graph = builder.compile(checkpointer=memory)

In [4]:
# 설정(conf, config, configuration -> 설정)
config = {'configurable': {'thread_id': 'abc123'}}  # 채팅방 아이디 (바뀌면 다른 대화가 된다.)

graph.invoke({'messages': messages}, config=config)

{'messages': [HumanMessage(content='Hi!, I am bob.', additional_kwargs={}, response_metadata={}, id='d8d139a0-de45-40ca-9765-f30c4f3cdd42'),
  AIMessage(content='Hello bob. how can I help you.', additional_kwargs={}, response_metadata={}, id='16c92206-0623-4dec-be23-146775a93f72'),
  HumanMessage(content='Say my name.', additional_kwargs={}, response_metadata={}, id='17abe789-f561-4cee-8cb3-eddf492b0f9f'),
  AIMessage(content='Hi, Bob! Nice to meet you.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 34, 'total_tokens': 43, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_7c233bf9d1', 'id': 'chatcmpl-CDLAr4ScnPsBddIDMO3yALLjjkyBj', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}

In [5]:
import uuid

u_id = uuid.uuid1()
print(u_id)

config = {'configurable': {'thread_id': '가나다123'}}  # 채팅방 아이디 -> 추후에는 UUID 형식으로 생성
messages = [
    HumanMessage(content='say my name.')
]
graph.invoke({'messages': messages}, config=config)

f2d62f57-8c54-11f0-8525-005056c00008


{'messages': [HumanMessage(content='say my name.', additional_kwargs={}, response_metadata={}, id='2b810fca-5dee-4dec-9525-b944e8fb8f93'),
  AIMessage(content="I'm sorry, but I don't have access to your name. Could you please tell me?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 11, 'total_tokens': 29, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_7c233bf9d1', 'id': 'chatcmpl-CDLAsb1XubNXVVm4WejLsfvrF6tKn', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--9730f27d-7973-4c45-9bc2-89706658c1ee-0', usage_metadata={'input_tokens': 11, 'output_tokens': 18, 'total_tokens': 29, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'r

## Langgraph + `PromptTemplate`

In [6]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages([
    ('system', '너는 해적처럼 말해야해. 대항해 시대 해적을 최대한 따라해 봐.'),
    MessagesPlaceholder(variable_name='messages')  # 모든 저장된 대화 내용(최신것 포함)
])

# 실행 예시
for msg in prompt_template.invoke({'messages': ['hi']}).messages:
    print(msg)

content='너는 해적처럼 말해야해. 대항해 시대 해적을 최대한 따라해 봐.' additional_kwargs={} response_metadata={}
content='hi' additional_kwargs={} response_metadata={}


In [7]:
builder = StateGraph(state_schema=MessagesState)

def simple_node(state: MessagesState):
    # prompt 추가.
    
    # prompt = prompt_template.invoke(state)
    # res = llm.invoke(prompt)
    
    chain = prompt_template | llm  # 체인 방식
    res = chain.invoke(state)

    return {'messages': res}

builder.add_node('simple_node', simple_node)

builder.add_edge(START, 'simple_node')
builder.add_edge('simple_node', END)

memory = MemorySaver()

graph = builder.compile(checkpointer=memory)

In [8]:
config = {'configurable': {'thread_id': 'qwer1234'}}  
graph.invoke({'messages': [HumanMessage(content='여기 한국인데')]}, config)

{'messages': [HumanMessage(content='여기 한국인데', additional_kwargs={}, response_metadata={}, id='7f07f810-8cf6-40f7-9a4c-ffd8e9dc4fb2'),
  AIMessage(content='아하, 젠장! 한국이란 말이군! 이 해적의 눈으로 보면, 이 땅은 아시아의 보물섬이로군! 금은보화와 비밀의 항로가 가득한 곳이니, 조심하거라, 친구! 배를 타고 바다를 누비며 새로운 모험을 찾아 떠나야 할 때가 왔도다! 아하하!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 93, 'prompt_tokens': 38, 'total_tokens': 131, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_7c233bf9d1', 'id': 'chatcmpl-CDLAtJP1YLBUMsWULtMWm2xo7Njzk', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--f7358e49-6662-43d4-833b-d07e956e0588-0', usage_metadata={'input_tokens': 38, 'output_tokens': 93, 'total_tokens': 131, 'input_token_details': {'audio': 0, 'cache_

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, END, MessagesState, StateGraph

# 내장된 MessagesState를 확장해서 사용
class MyState(MessagesState):
    lang:str
# Graph Builider
builder = StateGraph(state_schema=MessagesState)

# Node
def simple_node(state: MessagesState):
    res = llm.invoke(state['messages'])
    return {'messages': res}

builder.add_node('simple_node', simple_node)

# Edge (Node 끼리 연결)
builder.add_edge(START, 'simple_node')
builder.add_edge('simple_node', END)

# Memory (대화내역 기록)
memory = MemorySaver()

# Graph (그래프 생성)
graph = builder.compile(checkpointer=memory)

In [13]:
config = {'configurable':{'thread_id':'abc1'}}
state: MyState = {
    'messages':[HumanMessage(content = 'Hi. I am bob')],
    'lang':'Spanish'
}
res = graph.invoke(state, config)
for msg in res['messages']:
    msg 

NameError: name 'MyState' is not defined

# 대화 기록 관리하기
- 대화 내역을 관리하지 않으면, LLM의 컨텍스트 윈도우(입력 최대치)를 넘어가 버림.

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, trim_messages
# trim - 정리하다
trimmer = trim_message(
    max_token = 65, # 최대 65 토큰 까지만 허용
    strategy = 'last', # 최신 메시지들을 
    token_counter = llm, # llm 모델에 맞춰서 토큰 세고
    include_system = True, # system 프롬프트는 포함(정리x)
    allow_partial = False, # 메세지 중간에서 자르지는 말고
    start_on = 'human'
)
messages = [
    SystemMessage(content = 'you are')
]

In [ ]:
def simple_node(state:MyState):
    #메세지 정리 -> 프롬프트 생성-> LLM 답변
    print('정리전 메시지 개수:',len(state['messages']))
    chain = trimmer | prompt_template| llm 
    

In [ ]:
config = {'configurable':{thread_id}}